In [0]:
spark.conf.set(
    "fs.azure.account.key.storekm01.dfs.core.windows.net",
    'XXXXXXXXXXXXXXXXXXXXXXXXXX'
)

In [0]:
display(dbutils.fs.ls("abfss://mycon@storekm01.dfs.core.windows.net/"))

path,name,size,modificationTime
abfss://mycon@storekm01.dfs.core.windows.net/customer data.csv,customer data.csv,201666,1745328707000
abfss://mycon@storekm01.dfs.core.windows.net/product data.csv,product data.csv,924292,1745328707000
abfss://mycon@storekm01.dfs.core.windows.net/saledatecsv.csv,saledatecsv.csv,2655,1745334694000


In [0]:
%sql
CREATE DATABASE IF NOT EXISTS Sales

In [0]:
%sql
USE Sales

In [0]:
import pyspark
from pyspark.sql import SparkSession 
spark_session = SparkSession.builder.getOrCreate() 

In [0]:
df = spark_session.read.format( 
    "csv").option( 
    "header", True).load("abfss://mycon@storekm01.dfs.core.windows.net/saledatecsv.csv") 
df.show(2)

+----------+-------+----+------+----------+
|product_ID|  Month|Sale| Store|      Date|
+----------+-------+----+------+----------+
|      Rice|January| 455|StoreA|15-01-2020|
|      Rice|January| 455|StoreA|16-01-2020|
+----------+-------+----+------+----------+
only showing top 2 rows


In [0]:
# managed table
df.write.mode("overwrite").saveAsTable("Sales_table")

In [0]:
%sql
with firsttable as(
SELECT
product_ID,
sum(Sale) as revenue
FROM Sales_table
GROUP BY product_ID
ORDER BY revenue DESC),
secondtable as(
select 
*,
dense_rank()over(order by revenue desc) as rank_value
from firsttable)
select
MAX(case when rank_value=1 then product_ID end) as best_selling_product,
MAX(case when rank_value=5 then product_ID end) as worst_selling_product,
MAX(case when rank_value=1 then revenue end) as best_selling_product_price,
MAX(case when rank_value=5 then revenue end) as worst_selling_product_price,
MAX(case when rank_value=1 then revenue end)-MAX(case when rank_value=5 then revenue end) as difference
from secondtable

best_selling_product,worst_selling_product,best_selling_product_price,worst_selling_product_price,difference
Dal,Tomato,9165.0,5015.0,4150.0


In [0]:
# external table to exsisting SQL database
%python
jdbc_url = "jdbc:sqlserver://azsql.database.windows.net:1433;database=mydb"
connection_properties = {
  "user": "XXXXX",
  "password": "XXXXX",
  "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
df.write \
  .mode("overwrite") \
  .jdbc(
    url=jdbc_url, 
    table="Sales_table", 
    properties=connection_properties
  )
